In [1]:
from konlpy.tag import Okt
import pandas as pd
import torch
import torch.nn as nn
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np

from tqdm import tqdm

################## global variables ##################

FILTERS = "([~.,!?\"':;)(])"
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

path = '../../data_in/ChatBotData.csv'
vocab_path = '../../data_out/vocab.voc'
data_path = '../../data_in/ChatBotData.csv'
max_sequence_len = 25

############### end of global variables ##############


def load_data():
    '''
    1. Read Korean chatbot data(Q-A pairs) from the path, and put them into pd.DataFrame.
    2. split data into training set and validation set, and return them.
    
    @return
        - train_input: training question
        - train_label: training answer
        - test_input: test question
        - test_label: test answer
    '''
    data_df = pd.read_csv(path, header=0)
    question, answer = list(data_df['Q']), list(data_df['A'])
    train_input, test_input, train_label, test_label = train_test_split(question, answer, test_size=0.33, random_state=42)
    
    return train_input, train_label, test_input, test_label


# 사용 안 할 예정
def prepro_like_morphlized(data):
    '''
    @params:
        - data:
    '''
    # 형태소 분석 모듈 객체를 생성
    morph_analyzer = Okt()

    # 형태소 토크나이즈 결과 문장을 받을 리스트를 생성합니다.
    result_data = list()
    # 데이터에 있는 매 문장에 대해 토크나이즈를 할 수 있도록 반복문을 선언합니다.
    for seq in tqdm(data):
        # Okt.morphs 함수를 통해 토크나이즈 된 리스트 객체를 받고 다시 공백문자를 기준으로 문자열로 재구성 해줍니다.
        morphlized_seq = " ".join(morph_analyzer.morphs(seq))
        result_data.append(morphlized_seq)

    return result_data
    



# helper for load_vacabulary()
def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
            
    # 토그나이징과 정규표현식을 통해 만들어진 값들을 넘겨 준다.
    return [word for word in words if word]

# helper for load_vacabulary()
def make_vocabulary(vocabulary_list):
    # 리스트를 키가 단어이고 값이 인덱스인 딕셔너리를 만든다.
    char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
    # 리스트를 키가 인덱스이고 값이 단어인 딕셔너리를 만든다.
    idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
    # 두개의 딕셔너리를 넘겨 준다.
    return char2idx, idx2char


def load_vocabulary():
    # 사전을 담을 배열 준비한다.
    vocabulary_list = []

    # 사전을 구성한 후 파일로 저장 진행한다.
    # 그 파일의 존재 유무를 확인한다.
    if (not (os.path.exists(vocab_path))):
        # 이미 생성된 사전 파일이 존재하지 않으므로 데이터를 가지고 만들어야 한다.
        # 데이터가 존재 하면 사전을 만들기 위해서 데이터 파일의 존재 유무를 확인한다.
        if (os.path.exists(data_path)):
            # 데이터가 존재하면 pandas를 통해서 데이터를 불러오자
            data_df = pd.read_csv(data_path, encoding='utf-8')

            # 판다스의 데이터 프레임을 통해서 질문과 답에 대한 열을 가져 온다.
            question, answer = list(data_df['Q']), list(data_df['A'])

            data = []
            # 질문과 답변을 extend을 통해서 구조가 없는 배열로 만든다.
            data.extend(question)
            data.extend(answer)

            # 토큰나이져 처리 하는 부분이다.
            words = data_tokenizer(data)

            # set으로 중복이 제거된 집합을 생성한 후 리스트로 만들어 준다.
            words = list(set(words))

            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKNWON>"
            words[:0] = MARKER

        # 사전 리스트를 사전 파일로 만들어 넣는다.
        with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')


    # 사전 파일이 존재하면 여기에서 그 파일을 불러서 배열에 넣어 준다.
    with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())

    # 배열에 내용을 키와 값이 있는 딕셔너리 구조로 만든다.
    char2idx, idx2char = make_vocabulary(vocabulary_list)

    # 두가지 형태의 키와 값이 있는 형태를 리턴한다.
    # (예) 단어: 인덱스 , 인덱스: 단어)
    return char2idx, idx2char, len(char2idx)



def enc_processing(input_data, dictionary):
    '''
    @params
        - input_data: 인덱싱할 데이터 (train_input 또는 test_input)
        - dictionary: key(word)-value(index) pair
        
    @return
        - index sequences로 변환된 input_data의 words sequences
            + ex) [13042, 15055, 11881, 12337, 0, 0, ...]
    '''
    # 인덱스 값들을 가지고 있는 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    
    
    # 한줄씩 불어온다.
    for sequence in input_data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        
        # 하나의 문장을 인코딩 할때 가지고 있기 위한 배열이다.
        sequence_index = []

        # 문장을 스페이스 단위로 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])

            # 잘려진 단어가 딕셔너리에 존재 하지 않는 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > max_sequence_len:
            sequence_index = sequence_index[:max_sequence_len]

        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))

        # max_sequence_length보다 문장 길이가 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (max_sequence_len - len(sequence_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값을 sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)

    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 pytorch Variable에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_input_index), sequences_length





def dec_output_processing(input_data, dictionary):
    '''
    @params
        - input_data: 인덱싱할 데이터 (train_input 또는 test_input)
        - dictionary: key(word)-value(index) pair
        
    @return
        - index sequences로 변환된 words sequences
    '''
    
    # 인덱스 값들을 가지고 있는 배열이다.(누적된다)
    sequences_output_index = []
    
    # 하나의 디코딩 입력 되는 문장의 길이를 가지고 있다.(누적된다)
    sequences_length = []
    
    # 한줄씩 불어온다.
    for sequence in input_data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규식을 사용하여 필터에 들어 있는 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        
        # "하나의 문장"을 디코딩 할때 가지고 있기 위한 배열이다.
        sequence_index = []

        # 디코딩 입력의 처음에는 START(STD)가 와야 하므로 그 값을 넣어 주고 시작한다.
        # 문장에서 스페이스 단위별로 단어를 가져와서 딕셔너리의 값인 인덱스를 넣어 준다.
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > max_sequence_len:
            sequence_index = sequence_index[:max_sequence_len]

        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))

        # max_sequence_length보다 문장 길이가 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (max_sequence_len - len(sequence_index)) * [dictionary[PAD]]
        
        # 인덱스화 되어 있는 값을 sequences_output_index 넣어 준다.
        sequences_output_index.append(sequence_index)
    
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 pytorch Variable에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_output_index), sequences_length



def dec_target_processing(input_data, dictionary):
    '''
    @params
        - input_data: 인덱싱할 데이터 (train_input 또는 test_input)
        - dictionary: key(word)-value(index) pair
        
    @return
        - index sequences로 변환된 words sequences
    '''
    
    # 인덱스 값들을 가지고 있는 배열이다.(누적된다)
    sequences_target_index = []
    
    
    # input_data에서 한줄씩 불어온다.
    for sequence in input_data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규식을 사용하여 필터에 들어 있는 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)

        # 문장에서 스페이스 단위별로 단어를 가져와서 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 END 토큰을 넣어 준다
        if len(sequence_index) >= max_sequence_len:
            sequence_index = sequence_index[:max_sequence_len - 1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]

        # max_sequence_length보다 문장 길이가 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (max_sequence_len - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)

        
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 pytorch Variable에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index)




def pred_next_string(index_sequences, dictionary):
    '''
    @params
        - index_sequences: 인덱스로 표현된 sentences
            + ex) ex) [13042, 15055, 11881, 12337, 0, 0, ...] 와 같은 배열 여러 개
        - dictionary: key(index)-value(word) pair
    
    @return
        - answer: 단어들로 구성된 문장을 반환
        - is_finished: terminate condition
    '''
    
    # 텍스트 문장을 보관할 배열을 선언한다.
    sentence_string = []
    is_finished = False

    # 인덱스 배열 하나를 꺼내서 v에 넘겨준다.
    for v in index_sequences:
        # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
        sentence_string = [dictionary[index] for index in v]


    answer = ""
    # 패딩값도 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word == END:
            is_finished = True
            break

        if word != PAD and word != END:
            answer += word
            answer += " "

    # 결과를 출력한다.
    return answer, is_finished



def rearrange(input, output, target):
    features = {"input": input, "output": output}
    return features, target


def train_input_fn(train_input_enc, train_output_dec, train_target_dec, batch_size):
    '''
    학습에 사용될 mini-batches 구성하는 함수
    
    @params
        - train_input_enc:
        - train_output_dec:
        - train_target_dec:
        - batch_size:
        
    @return
    '''
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 각각 한 문장으로 자른다고 보면 된다.
    # train_input_enc, train_output_dec, train_target_dec 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((train_input_enc, train_output_dec, train_target_dec))

    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))

    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "train batchSize must not be None"

    # from_tensor_slices를 통해 나눈 것을 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)

    # 데이터 각 요소에 대해서 rearrange 함수를 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange)

    # repeat()함수에 원하는 에포크 수를 넣으면 해당 epoch 수 만큼 반복
    # 아무 인자도 없다면 무한으로 반복.
    dataset = dataset.repeat()

    # make_one_shot_iterator를 통해 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()

    # 이터레이터를 통해 다음 항목의 텐서 개체를 넘겨준다.
    return iterator.get_next()

In [2]:
morph_analyzer = Okt()

result_data = list()

data = list()
data.append('안녕 반가워')
data.append('심심하다 나랑 놀아줄 사람')
data.append('놀면 안돼 공부해야지')

for seq in tqdm(data):
    morph_seq = " ".join(morph_analyzer.morphs(seq))
    result_data.append(morph_seq)

100%|██████████| 3/3 [00:04<00:00,  1.43s/it]


In [3]:
print(data)
print(result_data)

['안녕 반가워', '심심하다 나랑 놀아줄 사람', '놀면 안돼 공부해야지']
['안녕 반가워', '심심하다 나 랑 놀아줄 사람', '놀면 안 돼 공부 해야지']


# make/load vocab

In [4]:
# 사전을 담을 배열 준비한다.
vocabulary_list = []

vocab_path = '../../data_out/vocab.voc'
data_path = '../../data_in/ChatBotData.csv'

# 사전을 구성한 후 파일로 저장 진행한다.
# 그 파일의 존재 유무를 확인한다.
if (not (os.path.exists(vocab_path))):
    # 이미 생성된 사전 파일이 존재하지 않으므로 데이터를 가지고 만들어야 한다.
    # 데이터가 존재 하면 사전을 만들기 위해서 데이터 파일의 존재 유무를 확인한다.
    if (os.path.exists(data_path)):
        # 데이터가 존재하면 pandas를 통해서 데이터를 불러오자
        data_df = pd.read_csv(data_path, encoding='utf-8')

        # 판다스의 데이터 프레임을 통해서 질문과 답에 대한 열을 가져 온다.
        question, answer = list(data_df['Q']), list(data_df['A'])

In [5]:
question[:10]

['12시 땡!',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'PPL 심하네',
 'SD카드 망가졌어',
 'SD카드 안돼',
 'SNS 맞팔 왜 안하지ㅠㅠ',
 'SNS 시간낭비인 거 아는데 매일 하는 중',
 'SNS 시간낭비인데 자꾸 보게됨']

In [6]:
data = []
# 질문과 답변을 extend을 통해서 구조가 없는 배열로 만든다.
data.extend(question)
data.extend(answer)

In [7]:
print(len(question))
print(len(answer))
print(len(data))

11823
11823
23646


In [8]:
words = []
for sentence in data:
    # FILTERS = "([~.,!?\"':;)(])"
    # 위 필터와 같은 값들을 정규화 표현식을 통해서 모두 "" 으로 변환 해주는 부분이다.
    sentence = re.sub(CHANGE_FILTER, "", sentence)
    for word in sentence.split():
        words.append(word)

In [9]:
words[:10]

['12시', '땡', '1지망', '학교', '떨어졌어', '3박4일', '놀러가고', '싶다', '3박4일', '정도']

In [10]:
words = list(set(words))

In [11]:
words[:10]

['끊기', '양심에', '되었길', '쌍커풀성형할까', '30이네', '어쩐지', '답은', '없을', '탈퇴', '쉬어']

In [12]:
# PAD = "<PADDING>"
# STD = "<START>"
# END = "<END>"
# UNK = "<UNKNWON>"
words[:0] = MARKER

In [13]:
words[:10]

['<PAD>',
 '<SOS>',
 '<END>',
 '<UNK>',
 '끊기',
 '양심에',
 '되었길',
 '쌍커풀성형할까',
 '30이네',
 '어쩐지']

In [14]:
# 사전 리스트를 사전 파일로 만들어 넣는다.
with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
    for word in words:
        vocabulary_file.write(word + '\n')

# 사전 파일이 존재하면 여기에서 그 파일을 불러서 배열에 넣어 준다.
with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
    for line in vocabulary_file:
        vocabulary_list.append(line.strip())

In [15]:
vocabulary_list[:10]

['<PAD>',
 '<SOS>',
 '<END>',
 '<UNK>',
 '끊기',
 '양심에',
 '되었길',
 '쌍커풀성형할까',
 '30이네',
 '어쩐지']

In [16]:
# 리스트를 키가 단어이고 값이 인덱스인 딕셔너리를 만든다.
char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}

# 리스트를 키가 인덱스이고 값이 단어인 딕셔너리를 만든다.
idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}

print(dict(list(char2idx.items())[:10]))
print(dict(list(idx2char.items())[:10]))

{'<PAD>': 0, '<SOS>': 1, '<END>': 2, '<UNK>': 3, '끊기': 4, '양심에': 5, '되었길': 6, '쌍커풀성형할까': 7, '30이네': 8, '어쩐지': 9}
{0: '<PAD>', 1: '<SOS>', 2: '<END>', 3: '<UNK>', 4: '끊기', 5: '양심에', 6: '되었길', 7: '쌍커풀성형할까', 8: '30이네', 9: '어쩐지'}


In [17]:
train_input, train_label, test_input, test_label = load_data()

In [18]:
print(train_input[:10])
print(train_label[:10])
print(test_input[:10])
print(test_label[:10])

['짝사랑 했던 여자가 떠나갔네.', '대리 불렀는데 안 오네', '내가 싫은 건 너도 싫어해줘서 고마워', '썸은 왜 타?', '짝녀 프사 자음이 궁금함.', '기다림이 습관이 됐나봐', '라면 먹고 싶어', '좋은 사람 있으면 소개시켜줘', '엄마가 내 방에 막 들어와', '애완동물 키우고 싶어']
['여기까지 인연이었나봅니다.', '잘했어요.', '싫은 게 통해야 편하죠.', '사귀기 전에 마음을 확인하는 단계라서 그렇지 않을까요?', '궁금할 수 있어요.', '좋은 분이시군요', '맛나게 끓여드세요.', '주변 사람들에게 부탁해보세요.', '엄마께 프라이버시를 지켜달라고 말해보세요.', '가족들과 상의해보세요.']
['죽을거 같네', '내일 시험이야', '정말.내 자신이 싫다', '이별후 네달째', '쌍커풀 해볼까', '내 생각 하나만 바꾸면 편할텐데.', '어떻게 살아가야 할까', '발 아파', '썸 타는 것도 귀찮아.', '좋아하는 애랑 전화하면']
['나쁜 생각 하지 마세요.', '컨디션 조절 하세요.', '자신은 사랑해주세요.', '바쁘게 살면서 잊어가요.', '눈은 기본이죠.', '하나 바꾸는 게 힘들지요.', '태연하게 시작해보세요.', '맥주병 같은 걸로 살살 문질러 주세요', '그냥 사귀세요.', '즐거울 거예요.']


In [20]:
# 인덱스 값들을 가지고 있는 배열이다.(누적된다.)
sequences_input_index = []
# 하나의 인코딩 되는 문장의 길이를 가지고 있다.(누적된다.)
sequences_length = []


# 한줄씩 불어온다.
for sequence in train_input:
    # FILTERS = "([~.,!?\"':;)(])"
    # 정규화를 사용하여 필터에 들어 있는 값들을 "" 으로 치환 한다.
    sequence = re.sub(CHANGE_FILTER, "", sequence)

    # 하나의 문장을 인코딩 할때 가지고 있기 위한 배열이다.
    sequence_index = []

    # 문장을 스페이스 단위로 자르고 있다.
    for word in sequence.split():
        # 잘려진 단어들이 딕셔너리에 존재하면 그 단어의 인덱스를 가져와 sequence_index에 추가한다.
        if char2idx.get(word) is not None:
            sequence_index.extend([char2idx[word]])

        # 잘려진 단어가 딕셔너리에 존재 하지 않는 경우 이므로 UNK(2)를 넣어 준다.
        else:
            sequence_index.extend([char2idx[UNK]])

    # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
    if len(sequence_index) > max_sequence_len:
        sequence_index = sequence_index[:max_sequence_len]

    # 하나의 문장에 길이를 넣어주고 있다.
    sequences_length.append(len(sequence_index))

    # max_sequence_length보다 문장 길이가 작다면 빈 부분에 PAD(0)를 넣어준다.
    sequence_index += (max_sequence_len - len(sequence_index)) * [char2idx[PAD]]

    # 인덱스화 되어 있는 값을 sequences_input_index에 넣어 준다.
    sequences_input_index.append(sequence_index)

In [27]:
print(sequences_input_index[0])
print(sequences_input_index[1])
print(sequences_input_index[2])
print(sequences_length[:10])

[13042, 15055, 11881, 12337, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[15415, 15431, 1894, 8651, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4332, 3560, 20157, 3047, 17563, 11445, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4, 4, 6, 3, 4, 3, 3, 4, 5, 3]
